<a href="https://colab.research.google.com/github/AnastasiiaDm/machine-learning/blob/main/DZ_17_time_series/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y numpy pmdarima
!pip install --upgrade numpy pmdarima darts

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pmdarima 2.0.4
Uninstalling pmdarima-2.0.4:
  Successfully uninstalled pmdarima-2.0.4
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (7.8 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (2.2 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from darts import TimeSeries


In [51]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/treasury.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,observation_date,T10Y2Y
0,2020-03-26,0.53
1,2020-03-27,0.47
2,2020-03-30,0.47
3,2020-03-31,0.47
4,2020-04-01,0.39


In [52]:
series = TimeSeries.from_dataframe(df, 'observation_date', 'T10Y2Y')
print(series)

<TimeSeries (DataArray) (observation_date: 1305, component: 1, sample: 1)> Size: 10kB
array([[[0.53]],

       [[0.47]],

       [[0.47]],

       ...,

       [[0.3 ]],

       [[0.35]],

       [[0.37]]])
Coordinates:
  * observation_date  (observation_date) datetime64[ns] 10kB 2020-03-26 ... 2...
  * component         (component) <U6 24B 'T10Y2Y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None


In [53]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values

# Fill missing values if needed
series = fill_missing_values(series)

# Split into train (80%) and test (20%)
train_size = int(0.8 * len(series))
train, test = series[:train_size], series[train_size:]

In [54]:
from darts.models import ExponentialSmoothing

# Initialize and train the model
model = ExponentialSmoothing()
model.fit(train)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
forecast = model.predict(len(test))

In [ ]:
plt.figure(figsize=(12, 6))
series.plot(label="Actual Data")
forecast.plot(label="Forecast", linestyle="dashed")
plt.legend()
plt.show()